In [1]:
import pandas as pd
import numpy as np
from pulp import *

In [2]:
# 4月のイベント
# *** は自由に変更可能、それ以外は期間中は固定

attri = 'クール'  # 属性
attri_diff = 5000  # 属性が違う時に下がる値
attri_diff_max = 0  # 属性が違うキャラ数の最大値 ***
# attri_poten_num = 131  # ポテンシャルスターの個数 ***
target_1 = 246600  # 攻
target_2 = 193200  # 守
target_3 = 166300  # 特攻
target_4 = 170600  # 特守
cost_min = 175  # コストの最小値
cost_max = 178  # コストの最大値
num_target = 7  # 人数の条件
num_max = 7  # 考慮する人数の最大値 ***
num_min = 7  # 考慮する人数の最小値 ***
num_diff = 20000  # 人数が違うときに下がる値
rare_need = 'SSR'  # 必須レア
rare_need_min = 2  # 必須レアの人数の最小値
# ur_lv_up = 1  # URのレベル上げの最大回数 ***
excel_name = 'aiwo_new_9.xlsx'

In [3]:
df = pd.read_excel(excel_name, header=None, skiprows=1)
a = df.values
N = a.shape[0]  # データの個数、すなわちaの行の数

number = a[:, 0].astype(np.int32)  # 番号：A列
name = a[:, 14]  # 名前：O列
f_name = a[:, 13]  # フル名前：N列
level = a[:, 7].astype(np.int32)  # レベル：H列
potential = a[:, 11]  # ポテンシャル：L列
rare = a[:, 10]  # レアリティ：K列
attribute = a[:, 16]  # 属性：Q列
cost = a[:, 17].astype(np.int32)  # コスト：R列
growth = a[:, 18]  # 成長型：S列

status_1 = a[:, 19].astype(np.int32)  # 現在攻：T列
status_2 = a[:, 20].astype(np.int32)  # 現在守：Y列
status_3 = a[:, 21].astype(np.int32)  # 現在特攻：V列
status_4 = a[:, 22].astype(np.int32)  # 現在特守：W列
status_sum = status_1 + status_2 + status_3 + status_4

#--------考えるもののラベルの作成-------#
# labelsに、レア度が'N'と'R'ではないものの、上からの番号を入れる。
labels = []
for i in range(N):
    if rare[i] != 'N' and rare[i] != 'R':
        labels.append(i)
ALL = len(labels)
print('今回考えるものの個数:', ALL)

#--------配列の抽出--------#
status_1_ = np.array([status_1[label] for label in labels])
status_2_ = np.array([status_2[label] for label in labels])
status_3_ = np.array([status_3[label] for label in labels])
status_4_ = np.array([status_4[label] for label in labels])
status_sum_ = np.array([status_sum[label] for label in labels])

number_ = np.array([number[label] for label in labels])
cost_ = np.array([cost[label] for label in labels])
level_ = np.array([level[label] for label in labels])
potential_ = np.array([potential[label] for label in labels])
rare_ = np.array([rare[label] for label in labels])
# ifとelseを使う場合は、forは後ろらしい。
rare_need_ = np.array([1 if (i == rare_need) else 0 for i in rare_])
# print(rare_need_)  # SSRなら1、N,R,SSR以外なら0

name_ = np.array([name[label] for label in labels])
f_name_ = np.array([f_name[label] for label in labels])
attribute_ = np.array([attribute[label] for label in labels])
attri_num = np.array([1 if (i != attri) else 0 for i in attribute_])
# print(attri_num)  # 属性が一致しないものなら1、一致すると0

NAME_ALL = len(set(name))
print('名前が重複していないキャラ数:', NAME_ALL)
name_list = list(set(name))  # set(name)は重複していないものの{  }型、listで[  ]型に変更
name_list = np.sort(name_list)  # あいうえお順
name_int = {n: i for i, n in enumerate(name_list)}  # 名前と数値を一対一対応する辞書
# print(name_int)
n_name_ = [name_int[na] for na in name_]  # n_name_は名前を表す数値、出席番号みたいな感じ
# print(n_name_)

matrix = np.zeros((NAME_ALL, ALL)).astype(np.int32)  # NAME_ALL*ALL次元の0行列
for i in range(NAME_ALL):
    for j in range(ALL):
        if n_name_[j] == i:
            matrix[i, j] = 1

今回考えるものの個数: 1237
名前が重複していないキャラ数: 111


In [4]:
# 保存するディレクトリ名
directory = './0602'  # このpythonファイルの名前
# 保存するファイル名
file_name = directory + '/nummin%d_nummax%d_attri%d' % (num_min, num_max, attri_diff_max)
print('保存するファイル名:', file_name)

保存するファイル名: ./0602/nummin7_nummax7_attri0


In [5]:
#--------問題の定義--------#
print('問題の定義なう...')
# m = LpProblem(sense=LpMaximize)  # 目的関数を最大化
m = LpProblem(name=file_name)  # 目的関数を最小化
x = [LpVariable('x%d' % i, cat=LpBinary)
     for i in range(ALL)]  # x:選ばれたら1、選ばれないものは0
z = LpVariable('z', cat='Integer')  # y:ステータスの差の絶対値
a = LpVariable('a', cat='Integer')  # a,b,c,d:記録用
b = LpVariable('b', cat='Integer')
c = LpVariable('c', cat='Integer')
d = LpVariable('d', cat='Integer')

# 目的関数:ステータスの差の絶対値の合計と、属性の差による誤差
m += z + attri_diff * lpDot(attri_num, x)
# 条件
m += lpDot(status_1_, x) - target_1 == a
m += lpDot(status_2_, x) - target_2 == b
m += lpDot(status_3_, x) - target_3 == c
m += lpDot(status_4_, x) - target_4 == d
m += a+b+c+d <= z
m += a+b+c-d <= z
m += a+b-c+d <= z
m += a+b-c-d <= z
m += a-b+c+d <= z
m += a-b+c-d <= z
m += a-b-c+d <= z
m += a-b-c-d <= z
m += -a+b+c+d <= z
m += -a+b+c-d <= z
m += -a+b-c+d <= z
m += -a+b-c-d <= z
m += -a-b+c+d <= z
m += -a-b+c-d <= z
m += -a-b-c+d <= z
m += -a-b-c-d <= z

for i in range(NAME_ALL):
    m += lpDot(matrix[i, :], x) <= 1  # それぞれの名前の番号の合計は最大1
m += lpSum(x[i] for i in range(ALL)) <= num_max
m += lpSum(x[i] for i in range(ALL)) >= num_min
m += lpDot(cost_, x) >= cost_min
m += lpDot(cost_, x) <= cost_max
m += lpDot(rare_need_, x) >= rare_need_min
m += lpDot(attri_num, x) <= attri_diff_max  # 属性が違うキャラ数の制約

問題の定義なう...


In [ ]:
#--------solve開始--------#
try:
    # ファイルを保存せずに終了
    # m.solve(PULP_CBC_CMD(msg=True))

    # ファイルを保存して終了
    os.makedirs(directory, exist_ok=True)  # ディレクトリが無ければ作成
    m.solve(PULP_CBC_CMD(msg=True, keepFiles=True, options=['mips', file_name+'-pulp.sol']))

except KeyboardInterrupt:
    m.solve(PULP_CBC_CMD(timeLimit=1, msg=False, options=['mips', file_name+'-pulp.sol']))


# 保存したファイルの結果確認用
# m.solve(PULP_CBC_CMD(timeLimit=1, msg=False, options=['mips', file_name+'-pulp.sol']))

In [8]:
#--------solve開始--------#
# try:
#     # ファイルを保存せずに終了
#     # m.solve(PULP_CBC_CMD(msg=True))

#     # ファイルを保存して終了
#     os.makedirs(directory, exist_ok=True)  # ディレクトリが無ければ作成
#     m.solve(PULP_CBC_CMD(msg=True, keepFiles=True, options=['mips', file_name+'-pulp.sol']))

# except KeyboardInterrupt:
#     m.solve(PULP_CBC_CMD(timeLimit=1, msg=False, options=['mips', file_name+'-pulp.sol']))


# 保存したファイルの結果確認用
# m.solve(PULP_CBC_CMD(timeLimit=1, msg=False, options=['mips', file_name+'-pulp.sol']))


# 時間制限を設けて実行
timeover = 5*60
m.solve(PULP_CBC_CMD(timeLimit=timeover))

In [7]:
#--------結果の表示--------#
moku = value(m.objective)  # 目的関数の値
kai = [i for i in range(ALL) if value(x[i]) == 1]  # value(x[i])は0か1。1のものの番号の配列

print('番号:', np.array([number_[i] for i in kai]))
print('名前:', np.array([f_name_[i] for i in kai]))
print('属性:', np.array([attribute_[i] for i in kai]))
print('レベル:', np.array([level_[i] for i in kai]))
print('レア:', np.array([rare_[i] for i in kai]))
print('現在のポテンシャル:', np.array([potential_[i] for i in kai]))
print('コスト: %d' % np.sum([cost_[i] for i in kai]),
      np.array([cost_[i] for i in kai]))
wa_1 = np.sum([status_1_[i] for i in kai])
wa_2 = np.sum([status_2_[i] for i in kai])
wa_3 = np.sum([status_3_[i] for i in kai])
wa_4 = np.sum([status_4_[i] for i in kai])
print('攻: %d (差:%d)' % (wa_1, target_1-wa_1),
      np.array([status_1_[i] for i in kai])-50*np.array([potential_[i] for i in kai]))
print('守: %d (差:%d)' % (wa_2, target_2-wa_2),
      np.array([status_2_[i] for i in kai])-50*np.array([potential_[i] for i in kai]))
print('特攻: %d (差:%d)' % (wa_3, target_3-wa_3),
      np.array([status_3_[i] for i in kai])-50*np.array([potential_[i] for i in kai]))
print('特守: %d (差:%d)' % (wa_4, target_4-wa_4),
      np.array([status_4_[i] for i in kai])-50*np.array([potential_[i] for i in kai]))
print('総合力:', np.array([status_sum_[i] for i in kai]) -
      200*np.array([potential_[i] for i in kai]))

abs_sum = np.abs(wa_1-target_1)+np.abs(wa_2-target_2) + \
    np.abs(wa_3-target_3)+np.abs(wa_4-target_4)  # 絶対値の合計
attri_sum = attri_diff*np.sum([attri_num[i] for i in kai])  # 属性違いによる減点

if len(kai) <= num_target:
    print('総評価:', np.int32(100000-abs_sum-attri_sum), end='')
    print(f' = 100000-{abs_sum}(絶対値の差)-{attri_sum}(属性違い)')
else:
    print('総評価:', np.int32(100000-abs_sum-attri_sum-num_diff), end='')
    print(f' = 100000-{abs_sum}(絶対値の差)-{attri_sum}(属性違い)-{num_diff}(人数オーバー)')

print('結果:', LpStatus[m.status])


番号: [ 349  352  567  642  644  896 1072]
名前: ['【瑞光祥春】天宮那智' '【アイオラ】ウミュウ' '【トップを狙う】白神七瀬' '【キミにプレゼント】雪城渚' '【スポーツ】葉山星名'
 '【ネコ娘ッ！】イリーナ・ロマネンコ' '【夜桜】愛葉礼華']
属性: ['クール' 'クール' 'クール' 'クール' 'クール' 'クール' 'クール']
レベル: [  1 110 110 110 100 100 130]
レア: ['SSR' 'UR' 'LR' 'UR' 'SSR' 'UR' 'LR']
現在のポテンシャル: [  0   0   0   0   0   0 100]
コスト: 178 [24 24 30 23 17 30 30]
攻: 246508 (差:92) [ 4580 33120 29250 44955 19404 38710 71489]
守: 193246 (差:-46) [ 5160 27855 32445 29970 15162 35035 42619]
特攻: 166215 (差:85) [ 7010 21150 58050 11340 19404 24605 19656]
特守: 170523 (差:77) [ 6430 22185 42885 13725 15162 28140 36996]
総合力: [ 23180 104310 162630  99990  69132 126490 170760]
総評価: 99700 = 100000-300(絶対値の差)-0(属性違い)
結果: Optimal
